In [8]:
import requests as r
import datetime as dt
import pandas as pd

# KiteUp - Icelandic weather alert system for kitesurfers

In [9]:

'''Constants
Weather Station Id's'''
REY = 1 # Reykjavík : Grótta
EYR = 1395 # Eyrarbakki : Eyrarbakki
GEL = 1480 # Geldingarnes : Geldingarnes
STR = 1473 # Straumsvík : Skógstjörn, Grótta


'''Configure your variables:
hour_range - Hour range, compared with <= and >=
wind_range - In m/s, <= and >=
min_rows - Minimum rows per day. Some forecasts only predict every 3 hours, Some every hour. compared with <= and >=
locations - Put in the stations you want to get updates about along with the directions you want to filter for
            Each weather station can have many kiteable locations you can check for, see STR(Straumsvík) station for example.
'''

# Time range
hour_range = ['09:00','15:00']
# Wind with m/s range that will be compared with <= and >=
# A 9 metre kite your wind range could be around [6,12] m/s
wind_range = [6,12]
check_days = 3 # How many days ahead to check, including today

include_directions = True
min_rows = 2 # Setting min rows as 2 because some forecasts only predict for every 3 hours

locations = {
                REY:[{'Grótta':['N','NNA','NV','VNV']}],
                EYR:[{'Ölfusárlón':['A','ASA','SSA','S','SSV','VSV']}],
                GEL:[{'Geldingarnes':['A','ASA','SSA','SA','V','VNV']}],
                STR:[{'Skógstjörn':['SV','SSV','VSV']}, 
                    {'Grótta':['N','NNA','NV','VNV']}]
            }



In [10]:
def query_weather_api():
    base_url = "https://apis.is"
    path = "/weather/forecasts/is/?stations="
    station_text_format = "{}," * len(locations)
    stations = station_text_format.format(*locations.keys())
    res = r.get(url=base_url + path + stations)
    return res
    '''
    Response:
    results - listi af dict:
        id - Stöðvanúmer
        name - Nafn á stöð
        atime - tími hvenær spá er gefin út væntanlega
        forecast - list af dict:
            það dict hefur ftime - tími á forminu %Y-%m-%d %H:%M:%S,
            F - vindhraði, 
            D - vindstefnu'''

In [11]:
def wind_filter(df):
    return df[(df.F  >= wind_range[0]) & (df.F <= wind_range[1])]

def directions_filter(df, directions):
    return df[df['D'].isin(directions)]

def hour_range_filter(df, hour_range):
    return df.between_time(*hour_range)

In [12]:

'''
day_check function:
Filters out if the forecast of the day has the right wind and kite–able directions
Parameters:
df – df containing weather for one day
location_id - weather station id '''
def day_check(df, location_id):
    for spot in locations[location_id]:
        for name, directions in spot.items():
            df = wind_filter(df)
            
            if not df.empty:
                if include_directions:
                    df = directions_filter(df, directions)
                if df.shape[0] >= min_rows:
                    print(spot)
                    print(df.head())


In [13]:
def main():
    query_response = query_weather_api()
    vedur = query_response.json()['results']
    for i in range(len(vedur)):
        location_id = int(vedur[i]['id'])
        df = pd.DataFrame(vedur[i]['forecast'])
        df['ftime'] = df['ftime'].astype('datetime64[s]')
        df['F'] = df['F'].astype('int')
        
        # Set the index to a DateTimeIndex so we can filter by hour
        df.set_index(pd.DatetimeIndex(df['ftime']), inplace=True)
        df = hour_range_filter(df, hour_range)
        # Get the first date
        day = df['ftime'].iloc[0]

        for i in range(check_days):
            df_day = df[df.ftime.dt.day == day.day]
            # Call day_check with a dataframe for each day
            day_check(df_day, location_id)
            # Iterate to the next day
            day += pd.DateOffset(day=i+1)


In [14]:
main()

{'Geldingarnes': ['A', 'ASA', 'SSA', 'SA', 'V', 'VNV']}
                       D  F    N    R   T  TD                      W  \
ftime                                                                  
2018-08-12 09:00:00    A  6  100  0.0  11   9  Lítils háttar rigning   
2018-08-12 11:00:00  ASA  6  100  0.3  11   9  Lítils háttar rigning   
2018-08-12 12:00:00  ASA  7  100  0.2  11   9  Lítils háttar rigning   
2018-08-12 13:00:00   SA  6  100  0.2  12  10  Lítils háttar rigning   
2018-08-12 14:00:00  ASA  6  100  0.0  13  10               Alskýjað   

                                  ftime  
ftime                                    
2018-08-12 09:00:00 2018-08-12 09:00:00  
2018-08-12 11:00:00 2018-08-12 11:00:00  
2018-08-12 12:00:00 2018-08-12 12:00:00  
2018-08-12 13:00:00 2018-08-12 13:00:00  
2018-08-12 14:00:00 2018-08-12 14:00:00  
